## 层次分析法

#### 1、 一致性检验

In [1]:
import numpy as np
# 生成一个4*4的矩阵
A = np.array([[1, 2, 3, 5], [1/2, 1, 1/2, 2],[1/3, 2, 1, 2], [1/5, 1/2, 1/2, 1]])
n = A.shape[0] # 获取矩阵的行数

# 计算最大特征值和特征向量
eig_val, eig_vec = np.linalg.eig(A)
Max_eig = max(eig_val)

# 计算一致性指标CI和随机一致性指标RI
CI = (Max_eig - n) / (n - 1)
RI = [0, 0.0001, 0.52, 0.89, 1.12, 1.26, 1.36, 1.41, 1.46, 1.49, 1.52, 1.54, 1.56, 1.58, 1.59]

# 计算一致性比CR
CR = CI / RI[n-1]

print("CI = ", CI)
print("RI = ", RI[n-1])

if CR < 0.1:
    print("CR = ", CR)
    print("矩阵A的一致性检验通过")
else:   
    print("CR = ", CR)
    print("矩阵A的一致性检验未通过")


CI =  (0.037610012730716846+0j)
RI =  0.9
CR =  (0.04178890303412983+0j)
矩阵A的一致性检验通过


#### 2、 算术平均法求权重

In [4]:
import numpy as np

A = np.array([[1, 2, 3, 5], [1/2, 1, 1/2, 2],[1/3, 2, 1, 2], [1/5, 1/2, 1/2, 1]])
# 计算矩阵A每一列的和
Asum = np.sum(A, axis=0)

# 计算标准化矩阵
n = A.shape[0]
Stand_A =  A / Asum

# 计算标准化矩阵每一行的和
Asumr = np.sum(Stand_A, axis=1)

# 计算权重
weight = Asumr / n

print("权重为：", weight)

权重为： [0.48885991 0.18192996 0.2318927  0.09731744]


#### 3、 几何平均法求求权重


In [6]:
import numpy as np

A = np.array([[1, 2, 3, 5], [1/2, 1, 1/2, 2],[1/3, 2, 1, 2], [1/5, 1/2, 1/2, 1]])

n = A.shape[0]

# 计算矩阵A每一列的乘积
prod_a = np.prod(A, axis=1)

# 计算几何平均值
prod_n_A = np.power(prod_a, 1/n)

# 归一化处理
re_prod_A = prod_n_A / np.sum(prod_n_A, axis=0)

# 计算权重
print("权重为：", re_prod_A)


array([30.        ,  0.5       ,  1.33333333,  0.05      ])

#### 4、 特征法求权重

In [7]:
import numpy as np

A = np.array([[1, 2, 3, 5], [1/2, 1, 1/2, 2],[1/3, 2, 1, 2], [1/5, 1/2, 1/2, 1]])

n = A.shape[0]

# 求出特征值和特征向量
eig_val, eig_vec = np.linalg.eig(A)

# 求出最大特征值的索引
max_index = np.argmax(eig_val)

# 找出对应的特征向量
max_vector = eig_vec[:, max_index]

# 归一化处理
weight = max_vector / np.sum(max_vector)

# 输出权重 
print("权重为：", weight)

权重为： [0.4933895 +0.j 0.17884562+0.j 0.230339  +0.j 0.09742588+0.j]


## TOPSIS法

In [11]:
import numpy as np

print("请输入参评数目")
n = int(input())
print("请输入指标数目")
m = int(input())

print('请输入类型矩阵, 1:极大型, 2:极小型, 3:中间型, 4:区间型')
kind = input().split(" ")

print("请输入矩阵：")
A = np.zeros(shape=(n, m)) # 初始化一个n*m的矩阵

for i in range(n):
    A[i] = input().split(" ") # 接受每行输入的数据
    A[i] = list(map(float, A[i])) # 将输入的数据转换为浮点数
print(f"输入矩阵为：\n{A}") # 输出输入的矩阵

def min2max(maxx, x):
    x = list(x)
    ans = [(maxx - e) for e in x] # 使用数组形式存储是由于下面有合并操作
    return np.array(ans)

def mid2max(bestx, x):
    x = list(x)
    h = [abs(e - bestx) for e in x] 
    M = max(h)
    if M == 0:
        M = 1
    ans = [[1 - e/M]  for e in h]
    return np.array(ans) 

def reg2max(lowx, highx, x):
    
    x = list(x)
    M = max(lowx - min(x), max(x) - highx)
    if M == 0:
        M = 1

    ans = []
    for i in range(len(x)):
        if x[i] < lowx:
            ans.append([1 - (lowx - x[i]) / M])
        elif x[i] > highx:
            ans.append([1 - (x[i] - highx) / M])
        else:
            ans.append([1])
            
    return np.array(ans)

X = np.zeros(shape=(n, 1))
for i in range(m):
    
    if kind[i] == '1':
        v = np.array(A[:, i])
    
    elif kind[i] == '2':
        maxA = max(A[:, i])
        v = min2max(maxA, A[:, i])
    
    elif kind[i] == '3':
        print("类型三, 请输入最佳值：")
        bestA = eval(input())
        v = mid2max(bestA, A[:, i])
    
    elif kind[i] == '4':
        print("类型四, 请输入区间[a, b]值a:")
        lowA = eval(input())
        print("类型四, 请输入区间[a, b]值b:")
        highA = eval(input())
        v = reg2max(lowA, highA, A[:, i])
        
    if i == 0:
        X = v.reshape(-1, 1)
    else:
        X = np.hstack([X, v.reshape(-1, 1)]) # 横向合并

print(f"统一指标后的矩阵为：\n{X}")

X = X.astype('float')
for j in range(m):
    X[:, j] = X[:, j] / np.sqrt(sum(X[:, j]**2))

print(f"标准化矩阵为：\n{X}")

x_max = np.max(X, axis=0)
x_min = np.min(X, axis=0)

# 计算距离 m维空间中两点之间的距离 n代表方案数
d_z = np.sqrt(np.sum(np.square((X - np.tile(x_max, (n, 1)))), axis=1))
d_f = np.sqrt(np.sum(np.square((X - np.tile(x_min, (n, 1)))), axis=1))

print(f"每个指标的最大值{x_max}")
print(f"每个指标的最小值{x_min}")
print(f"到最大值的距离{d_z}")
print(f"到最小值的距离{d_f}")

s = d_f / (d_f + d_z) # 计算综合得分
Score = 100 * s / sum(s) # 计算综合得分百分比
for i in range(len(Score)):
    print(f"第{i+1}个方案的得分为{Score[i]}") 

请输入参评数目
请输入指标数目
请输入类型矩阵, 1:极大型, 2:极小型, 3:中间型, 4:区间型
请输入矩阵：
输入矩阵为：
[[  9.  10. 175. 120.]
 [  8.   7. 164.  80.]
 [  6.   3. 157.  90.]]
类型三, 请输入最佳值：
类型四, 请输入区间[a, b]值a:
类型四, 请输入区间[a, b]值b:
统一指标后的矩阵为：
[[9.  0.  0.  0. ]
 [8.  3.  0.9 0.5]
 [6.  7.  0.2 1. ]]
标准化矩阵为：
[[0.66896473 0.         0.         0.        ]
 [0.59463532 0.3939193  0.97618706 0.4472136 ]
 [0.44597649 0.91914503 0.21693046 0.89442719]]
每个指标的最大值[0.66896473 0.91914503 0.97618706 0.89442719]
每个指标的最小值[0.44597649 0.         0.         0.        ]
到最大值的距离[1.61175952 0.69382053 0.79132442]
到最小值的距离[0.22298824 1.15334862 1.30072534]
第1个方案的得分为8.886366735657832
第2个方案的得分为45.653341055701134
第3个方案的得分为45.46029220864103


## 熵权法

In [3]:
import numpy as np
def mylog(p): # 自定义log函数，避免log(0)的情况
    n = len(p)
    lnp = np.zeros(n)
    for i in range(n):
        if p[i] == 0:
            lnp[i] = 0
        else:
            lnp[i] = np.log(p[i])
    return lnp 


X = np.array([[9, 0, 0, 0], [8, 3, 0.9, 0.5], [6, 7, 0.2, 1]])

# 进行标准化处理 
Z = X / np.sqrt(np.sum(X**2, axis=0))

print(f"标准化矩阵 Z = \n{Z}")

n, m = Z.shape # 获取矩阵的行数和列数
D = np.zeros(m) # 用于保存每个指标的信息效用值

for i in range(m):
    x = Z[:, i] # 获取第i列的数据
    p = x / np.sum(x)  # 对第i个指标进行归一化处理得到概率分布

    e = -np.sum(p * mylog(p)) / np.log(n) # 计算信息熵
    D[i] = 1 - e # 计算信息效用值 信息熵越大，信息效用值越小 全部数据一样时，信息熵为1

W = D / np.sum(D) # 计算权重

print(f"权重 W = {W}")



标准化矩阵 Z = 
[[0.66896473 0.         0.         0.        ]
 [0.59463532 0.3939193  0.97618706 0.4472136 ]
 [0.44597649 0.91914503 0.21693046 0.89442719]]
权重 W = [0.00856537 0.30716152 0.39326471 0.2910084 ]


## 模糊综合评价

In [13]:
import numpy as np
# 单因素评价系数由专家评分法或隶属函数得到
# 一级模糊综合评价
# 影响运行费用的各因素的单因素评价矩阵为 （隶属度的集合）
R23 = np.array([
    [0.18, 0.14, 0.18, 0.14, 0.13, 0.23],
    [0.15, 0.20, 0.15, 0.25, 0.13, 0.15],
    [0.25, 0.12, 0.13, 0.12, 0.18, 0.20],
    [0.20, 0.13, 0.18, 0.17, 0.19, 0.13],
    [0.22, 0.11, 0.12, 0.18, 0.13, 0.24],
    [0.17, 0.16, 0.13, 0.25, 0.18, 0.11]
])
# 权重分配为
A23 = np.array([0.20, 0.15, 0.10, 0.20, 0.15, 0.10])
# 评价结果
# np.dot()函数对于一维数组是点积，对于二维数组是矩阵乘法
B23 = np.dot(A23, R23)

# 2. 二级模糊综合评价
# 产品情况的二级评价矩阵

R1 = np.array([
    [0.12, 0.18, 0.17, 0.23, 0.17],
    [0.15, 0.10, 0.15, 0.20, 0.14],
    [0.20, 0.12, 0.13, 0.18, 0.25],
    [0.13, 0.13, 0.19, 0.16, 0.13],
    [0.17, 0.13, 0.25, 0.18, 0.16]
])

A1 = np.array([0.15, 0.40, 0.25, 0.10, 0.10])

B1 = np.dot(A1, R1)

# 销售能力的二级评价矩阵
R2 = np.array([
    [0.13, 0.15, 0.14, 0.18, 0.16, 0.25],
    [0.12, 0.16, 0.13, 0.17, 0.19, 0.23],
    B23,
    [0.14, 0.13, 0.15, 0.16, 0.18, 0.24],
    [0.16, 0.15, 0.15, 0.17, 0.18, 0.19]
])

A2 = np.array([0.2, 0.15, 0.25, 0.25, 0.15])
B2 = np.dot(A2, R2)

# 市场需求的二级评判
R3 = np.array([
    [0.15, 0.14, 0.13, 0.18, 0.14, 0.26],
    [0.16, 0.15, 0.18, 0.14, 0.16, 0.21]
])
A3 = np.array([0.55, 0.45])
B3 = np.dot(A3, R3)

print(B1)
print(B2)
print(B3)

# 三级模糊综合评判
import numpy as np
# 获取最大长度
max_len = max(len(B1), len(B2), len(B3))

# 使用 `np.pad` 将所有数组填充到相同的长度
B1_padded = np.pad(B1, (0, max_len - len(B1)), 'constant')
B2_padded = np.pad(B2, (0, max_len - len(B2)), 'constant')
B3_padded = np.pad(B3, (0, max_len - len(B3)), 'constant')

# 计算加权平均
R = np.array([B1_padded, B2_padded, B3_padded])
A = np.array([0.4, 0.3, 0.3])
weighted_sum = np.dot(A, R)
print(weighted_sum)



[0.158  0.123  0.162  0.1935 0.173 ]
[0.146375 0.141125 0.142125 0.167875 0.16725  0.213375]
[0.1545 0.1445 0.1525 0.162  0.149  0.2375]
[0.1534625 0.1348875 0.1531875 0.1763625 0.164075  0.1352625]


## 灰色关联分析

In [ ]:
# 请输入初始矩阵[[55, 24, 10, ]]